In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.util import deprecations
from sqlalchemy.sql import text
deprecations.SILENCE_UBER_WARNING = True
# create database connection
import psycopg2
import psycopg2.extras
from typing import Iterator, Dict, Any

def insert_execute_batch(ssql, df_dict) -> None:
    
    connection = psycopg2.connect(
        host="98.98.117.105",
        port='5432',
        database="medols",
        user="postgres",
        password='FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    )

    try:
        connection.autocommit = True
        with connection.cursor() as cursor:
            psycopg2.extras.execute_batch(cursor, ssql, df_dict)

    except:
        # Rollback changes if there's an error
        print(f"Error executing query: {e}")

    finally:
        connection.close()

def execute_query_psql(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            # print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()


In [2]:
from datetime import datetime
import re
import logging

# Configure logging
logging.basicConfig(filename='/home/jup_user/multipool/ai-case-study/date_parsing_errors.log', level=logging.ERROR, format='%(asctime)s:%(levelname)s:%(message)s')

from datetime import datetime

def convert_to_python_datetime(date_str):
    # Mapping Indonesian day names to English
    days_ind_to_eng = {
        'Senin': 'Monday',
        'Selasa': 'Tuesday',
        'Rabu': 'Wednesday',
        'Kamis': 'Thursday',
        'Jumat': 'Friday',
        'Sabtu': 'Saturday',
        'Minggu': 'Sunday'
    }

    # Indonesian month names to English
    months_ind_to_eng = {
        'Januari': 'January',
        'Februari': 'February',
        'Maret': 'March',
        'April': 'April',
        'Mei': 'May',
        'Juni': 'June',
        'Juli': 'July',
        'Agustus': 'August',
        'September': 'September',
        'Oktober': 'October',
        'November': 'November',
        'Desember': 'December'
    }

    try:
        print(date_str)
        date_str = date_str.replace("Jum'at", "Jumat")

        # Split the date string and trim whitespace around elements
        parts = [part.strip() for part in date_str.split(',')]
        day_part = parts[0].split()[0]  # Split "Kamis," and take "Kamis"
        date_and_month = parts[1].split()  # Split "02 Mei  2024 06:01 WIB"
        
        # Replace Indonesian day and month names with English equivalents
        day_eng = days_ind_to_eng[day_part]
        month_ind = date_and_month[1]
        month_eng = months_ind_to_eng[month_ind]
        print(date_and_month)
        
        # check for correct pattern
        if (date_and_month[3] != '-'):
            # Construct the cleaned and translated date string
            cleaned_date_str = f"{day_eng}, {date_and_month[0]} {month_eng} {date_and_month[2]} {date_and_month[3]} WIB"

        else:
            # Construct the cleaned and translated date string
            cleaned_date_str = f"{day_eng}, {date_and_month[0]} {month_eng} {date_and_month[2]} {date_and_month[4]} WIB"
        
        print(cleaned_date_str)
        
        # Define the format of the incoming date string
        date_format = '%A, %d %B %Y %H:%M WIB'
    
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(cleaned_date_str, date_format)

        return date_obj

    except Exception as e:
        # print(e)
        return None
            
def parse_date_to_datetime(date_str):
    # Regex pattern to extract the date, month, year, and time from the string
    pattern = r"\w+\s+(\d{1,2})\s+(\w+)\s+(\d{4})\s+-\s+(\d{2}:\d{2})\s+WIB"
    match = re.search(pattern, date_str)

    try:
        if match:
            day, month, year, time = match.groups()
            day, year = int(day), int(year)
    
            # Map months from Bahasa Indonesia to English for datetime compatibility
            month_mapping = {
                'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4, 'Mei': 5, 'Juni': 6,
                'Juli': 7, 'Agustus': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
            }
            
            month = month_mapping.get(month, None)
            if month is None:
                return f"Error: Month '{month}' is not recognized."
    
            # Convert date and time into a datetime object
            hour, minute = map(int, time.split(':'))
            date_time = datetime(year, month, day, hour, minute)
    
            return date_time
        else:
            # print("parse_date_to_datetime %s" % date_str)
            return None

    except Exception as e:
        logging.error(f"Original Data: {date_str}, ID: {row.get('id', 'Unknown ID')}")
        return None

def parse_bahasa_date_enhanced(row):
    # Extended month mappings including abbreviations for Bahasa Indonesia and English
    month_mapping_enhanced = {
        'Januari': 1, 'Jan': 1, 'January': 1,
        'Februari': 2, 'Feb': 2, 'February': 2,
        'Maret': 3, 'Mar': 3, 'March': 3,
        'April': 4, 'Apr': 4,
        'Mei': 5, 'May': 5,
        'Juni': 6, 'Jun': 6, 'June': 6,
        'Juli': 7, 'Jul': 7, 'July': 7,
        'Agustus': 8, 'Ags': 8, 'Aug': 8, 'August': 8,
        'September': 9, 'Sep': 9,
        'Oktober': 10, 'Okt': 10, 'Oct': 10,
        'November': 11, 'Nov': 11,
        'Desember': 12, 'Des': 12, 'Dec': 12, 'December': 12
    }
    date_str = row['publish_date']
    # Clean up the string by removing vertical bars and commas
    date_str = date_str.replace('|', ' ').replace(',', ' ').strip()
    # Replace "Jum'at" with "Jumat"
    date_str = date_str.replace("Jum'at", "Jumat")

    try:
        # Regex patterns to capture different date formats
        match = re.search(r"(\d{1,2})[\s/-](\d{1,2}|[a-zA-Z]+)[\s/-](\d{4})\s+(\d{2}):(\d{2})", date_str)
        if match:
            # print('here1')
            day, month, year, hour, minute = match.groups()
            if month.isdigit():
                month = int(month)
            else:
                month = month_mapping_enhanced.get(month.capitalize())
                if month is None:
                    raise ValueError(f"Unknown month: {month}")

            # Create datetime object and format it
            dt = datetime(int(year), month, int(day), int(hour), int(minute))
            # print(dt)
            return dt.strftime('%Y-%m-%d %H:%M:%S'), row['id']
        else:
            # further combination
            conv_date_time = parse_date_to_datetime(row['publish_date'])
            if conv_date_time is None:
                # print('here2')
                conv_date_time2 = convert_to_python_datetime(row['publish_date'])

                return conv_date_time2, row['id']
            else:
                print('here3')
                return conv_date_time, row['id']
                # print('here5')
                
    except Exception as e:
        logging.error(f"Original Data: {date_str}, ID: {row.get('id', 'Unknown ID')}")
        print(f"Error Msg: {e}, Original Data: {date_str}, ID: {row.get('id', 'Unknown ID')}")
        return None


In [3]:
sql = """
select * from news_national_medol where calculation_mark is null
"""

df_source = execute_query_psql(sql)

In [4]:
def update_data_sql(row):
    if row[0] is None:
        ssql = """
        update news_national_medol 
        set    calculation_mark = 1
        where id = %s        
        """ % (row[1])
        execute_query_psql(ssql,)

    else:    
        ssql = """
        update news_national_medol 
        set    publish_date_convert = '%s', 
               calculation_mark = 1
        where id = %s
        """ % (row[0],row[1])
        execute_query_psql(ssql,)
        
# print(len(df_source))
if len(df_source) > 0:
    df_res = df_source.apply(parse_bahasa_date_enhanced, axis=1)
    df_res.apply(update_data_sql)